**Statistical site data over all days**

Reads:
* Regular input csv for all days, without gps

Outputs:
* '../static/data/sites/sites_behavior.json'

The output json contains many information on overall visitor behavior per site, such as total trips, total users, average trip duration, favorite transport mode, other sites visited by users and previous site visited before this one


TODO: some cleanup / documentation needed


In [ ]:
# Configuration
OUTPUT_SITE_BEHAVIOR_FILE = '../static/data/sites/sites_behavior.json'

In [ ]:
import pandas as pd
import geopandas as gpd
import h3pandas
from shapely.geometry import Point, Polygon, LineString
import json
import folium
import os
import numpy as np

In [ ]:
li = []
li.append(pd.read_csv("sources/data_france_2024-07-24.csv"))
li.append(pd.read_csv("sources/data_france_2024-07-25.csv"))
li.append(pd.read_csv("sources/data_france_2024-07-26.csv"))
li.append(pd.read_csv("sources/data_france_2024-07-27.csv"))
li.append(pd.read_csv("sources/data_france_2024-07-28.csv"))
li.append(pd.read_csv("sources/data_france_2024-07-29.csv"))
li.append(pd.read_csv("sources/data_france_2024-07-30.csv"))
li.append(pd.read_csv("sources/data_france_2024-07-31.csv"))
li.append(pd.read_csv("sources/data_france_2024-08-01.csv"))
# li.append(pd.read_csv("sources/data_france_2024-08-02.csv"))
# li.append(pd.read_csv("sources/data_france_2024-08-03.csv"))
# li.append(pd.read_csv("sources/data_france_2024-08-04.csv"))
df_src = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
df_src = df_src.rename(columns={"moover_id": "user_id"})

In [ ]:
tr = {
-10 : "NOT_DEFINED",
0 : "UNKNOWN",
1 : "PASSENGER_CAR",
2 : "MOTORCYCLE",
3 : "HEAVY_DUTY_VEHICLE",
4 : "BUS",
5 : "COACH",
6 : "RAIL_TRIP",
7 : "BOAT_TRIP",
8 : "BIKE_TRIP",
9 : "PLANE",
10 : "SKI",
11 : "FOOT",
12 : "IDLE",
13 : "OTHER",
101 : "SCOOTER",
102 : "HIGH_SPEED_TRAIN"
}
df_src['transportation_mode_tr'] = df_src['transportation_mode'].apply(lambda x: tr[x])

In [ ]:
df = df_src.groupby("journey_id").agg(
    journey_starting_longitude=('starting_longitude', 'first'),
    journey_starting_latitude=('starting_latitude', 'first'),
    journey_ending_longitude=('ending_longitude', 'last'),
    journey_ending_latitude=('ending_latitude', 'last'),
    start_time=('start_time', 'first'),
    end_time=('end_time', 'last'),
    user_id=('user_id', 'last')
).reset_index()
df

In [ ]:
# Convert columns to datetime
df['start_time'] = pd.to_datetime(df['start_time'], format="mixed")
df['end_time'] = pd.to_datetime(df['end_time'], format="mixed")

# Calculate duration in seconds
df['duration'] = (df['end_time'] - df['start_time']).dt.total_seconds()

In [ ]:
dfh3 = df.h3.geo_to_h3(9, lat_col="journey_starting_latitude", lng_col="journey_starting_longitude", set_index=False)
dfh3["start_h3"] = dfh3["h3_09"]
dfh3 = dfh3.h3.geo_to_h3(9, lat_col="journey_ending_latitude", lng_col="journey_ending_longitude", set_index=False)
dfh3["end_h3"] = dfh3["h3_09"]

In [ ]:
# h3 level 9 cells loosely covering every olympic location
# For bigger sites, we try to cover the whole site with many cells
# There might be imprecisions:
# * With people parking far outside the cell (even though the trip should continue with a walking segment)
# * With people stopping in the cell, but not going to the Olympic location (very true for locations near train stations)
# * With people moving within the cell

SITES_OLYMPIQUES = {
    #"saint_lazare": {"h3": ["891fb475b37ffff"]},
    #"gare_du_nord": {"h3": ["891fb4660dbffff"]},
    #"gare_de_lest": {"h3": ["891fb466053ffff", "891fb4660cfffff", "891fb4660cbffff"]},
    "addidas_arena": {"name": "Arena Porte de La Chapelle", "h3": ["891fb4664afffff", "891fb4664abffff", "891fb466433ffff"]},
    "pont_alexandre_trois": {"name": "Pont Alexandre III", "h3": ["891fb4675d3ffff"]},
    "invalides": {"name": "Invalides", "h3": ["891fb4675dbffff"]},
    "grand_palais": {"name": "Grand Palais", "h3": ["891fb475b6bffff"]},
    "tour_eiffel": {"name": "Stade Tour Eiffel", "h3": ["891fb46741bffff", "891fb467413ffff", "891fb467403ffff", "891fb46740bffff"]},
    "trocadero": {"name": "Trocadéro", "h3": ["891fb4674d7ffff"]},
    "grand_palais_ephemere": {"name": "Arena Champ-de-Mars", "h3": ["891fb467477ffff"]},
    "place_concorde": {"name": "La Concorde", "h3": ["891fb46759bffff"]},
    "porte_versailles": {"name": "Arena Paris Sud", "h3": ["891fb467673ffff"]},
    "arena_bercy": {"name": "Arena Bercy", "h3": ["891fb46440fffff"]},
    "parc_des_princes": {"name": "Parc des Princes", "h3": ["891fb462b8bffff","891fb462b8fffff", "891fb462b13ffff"]},
    "rolland_garros": {"name": "Stade Roland-Garros", "h3": ["891fb462867ffff"]},
    "la_defense_arena": {"name": "Paris La Défense Arena", "h3": ["891fb475313ffff", "891fb47538fffff"]},
    "stade_de_france": {"name": "Stade de France", "h3": ["891fb474b83ffff", "891fb474b9bffff", "891fb474b93ffff", "891fb474b97ffff", "891fb474b87ffff"]},
    "villepinte" : {"name": "Arena Paris Nord", "h3": ["891fb428197ffff", "891fb42aa5bffff" , "891fb42aa4bffff", "891fb42aa43ffff", "891fb42aa57ffff", "891fb4281b3ffff", "891fb42aa4fffff", "891fb42aa47ffff", "891fb42aa0bffff", "891fb4281b7ffff", "891fb42aa7bffff", "891fb42aa73ffff", "891fb42aa0fffff", "891fb42aa6bffff", "891fb42aa63ffff", "891fb42aa77ffff", "891fb42aa3bffff", "891fb42aa67ffff", "891fb42aa2bffff"]},
    "centre_aquatique_st_denis" : {"name": "Centre Aquatique", "h3": ["891fb474b9bffff"]},
    "tir_chateauroux" : {"name": "Centre National de Tir de Châteauroux", "h3": ["89186dd5027ffff", "89186dd51cbffff", "89186dd515bffff", "89186dd5153ffff", "89186dd51cfffff", "89186dd514bffff", "89186dd5143ffff", "89186dd5157ffff", "89186dd514fffff", "89186dd5147ffff", "89186dd510bffff", "89186dd5173ffff", "89186dd510fffff"]},
    "chateau_de_versailles" : {"name": "Château de Versailles", "h3": ["891fb4632d3ffff", "891fb4632c3ffff", "891fb4632d7ffff", "891fb4632c7ffff"]},
    "colline_elancourt" : {"name": "Colline d'Elancourt", "h3": ["891fb4782afffff", "891fb478237ffff", "891fb478233ffff", "891fb4782abffff", "891fb478207ffff", "891fb47823bffff", "891fb4783c3ffff"]},
    "hotel_de_ville" : {"name": "Hôtel de Ville", "h3": ["891fb466257ffff"]},
    "golf_national" : {"name": "Golf National", "h3": ["891fb46a437ffff", "891fb46a5cbffff", "891fb46a5c3ffff", "891fb46a5cfffff", "891fb46a51bffff", "891fb46a503ffff"]},
    "marina_marseille" : {"name": "Marina de Marseille", "h3": ["89396802127ffff", "89396802c5bffff", "8939680212fffff", "89396802e97ffff", "89396802c4bffff"]},
    "bourget" : {"name": "Site d'escalade Bourget", "h3": ["891fb429437ffff", "891fb429423ffff", "891fb429427ffff"]},
    "saint_quentin" : {"name": "Saint-Quentin-en-Yvelines", "h3": ["891fb478e37ffff", "891fb4788cbffff", "891fb4788cfffff", "891fb478e23ffff", "891fb478e27ffff", "891fb478853ffff"]},
    "bordeaux" : {"name": "Stade de Bordeaux", "h3": ["89186b6c553ffff", "89186b6c5cfffff", "89186b6c557ffff", "89186b6c51bffff", "89186b6c5cbffff", "89186b6c5c3ffff"]},
    "nantes" : {"name": "Stade de la Beaujoire", "h3": ["8918458458fffff", "89184584587ffff", "89184584583ffff", "89184584597ffff", "89184586e4bffff"]},
    "lyon" : {"name": "Stade de Lyon", "h3": ["891f9025dafffff", "891f9024adbffff", "891f9025d33ffff", "891f9025d37ffff", "891f9024acbffff", "891f9025d23ffff", "891f9025d27ffff", "891f9024a5bffff"]},
    "stade_marseille" : {"name": "Stade de Marseille", "h3": ["89396802c2bffff", "89396802c2fffff", "89396802893ffff", "89396802897ffff"]},
    "nice" : {"name": "Stade de Nice", "h3": ["893969a001bffff", "893969a0057ffff", "893969a000bffff", "893969a0003ffff"]},
    "saint_etienne" : {"name": "Stade Geoffroy-Guichard", "h3": ["891f9060cc7ffff", "891f9060ccfffff", "891f9060c1bffff"]},
    "vaires_sur_marne" : {"name": "Stade Nautique de Vaires-sur-Marne", "h3": ["891fb092e77ffff", "891fb092e2fffff", "891fb09285bffff", "891fb092e63ffff", "891fb092e67ffff", "891fb092a93ffff", "891fb092a97ffff", "891fb092e6fffff", "891fb092a9bffff", "891fb092a83ffff", "891fb092ad7ffff", "891fb092a8bffff", "891fb092a8fffff", "891fb092abbffff"]},
    "lille" : {"name": "Stade Pierre Mauroy", "h3": ["89194d28ac7ffff", "89194d28a8bffff", "89194d28a83ffff", "89194d28a1bffff", "89194d28a13ffff", "89194d28a8fffff", "89194d28a03ffff", "89194d28a17ffff"]},
    "colombes" : {"name": "Stade Yves-du-Manoir", "h3": ["891fb4742c7ffff", "891fb474213ffff", "891fb4742cfffff", "891fb47421bffff", "891fb474203ffff", "891fb474257ffff", "891fb47420bffff"]},
    #"tahiti" : {"name": "Teahupo'o, Tahiti", "h3": []}
}

In [ ]:
# Create an inverted dictionary for fast lookups
h3_to_site = {}
for site, info in SITES_OLYMPIQUES.items():
    for h3 in info["h3"]:
        h3_to_site[h3] = info["name"]

In [ ]:
# Define the optimized find_site function
def find_site(h3cell):
    return h3_to_site.get(h3cell, None)
dfh3["start_site"] = dfh3["start_h3"].apply(find_site)
dfh3["end_site"] = dfh3["end_h3"].apply(find_site)

In [ ]:
df_filtered = dfh3[dfh3["end_site"].notna()][dfh3["end_site"] != dfh3["start_site"]]
df_filtered

In [ ]:
# Sort the DataFrame by user_id and end_time
df_filtered = df_filtered.sort_values(by=['user_id', 'end_time'])

# Create the previous_site and previous_end_time columns
df_filtered['previous_site'] = df_filtered.groupby('user_id')['end_site'].shift(1)
df_filtered['previous_end_time'] = df_filtered.groupby('user_id')['end_time'].shift(1)

# Calculate time_since_previous_site
df_filtered['time_since_previous_site'] = df_filtered['end_time'] - df_filtered['previous_end_time']

In [ ]:
df_filtered['previous_site'] = df_filtered['previous_site'].fillna("Aucun")

In [ ]:
df_filtered

In [ ]:
gp = df_filtered.groupby(["end_site", "previous_site"]).agg(TotalOccurence=("journey_id", "count"), UniqueUsers=("user_id", "nunique"), MeanDuration=("time_since_previous_site", "mean")).sort_values(by=['end_site', 'TotalOccurence'])
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(gp)
gp

In [ ]:
df_filter = dfh3[dfh3["end_site"].notna()]
df_filter = df_filter[df_filter["end_site"] != df_filter["start_site"]]
#display(df_filter[df_filter["end_site"] == "Arena Paris Nord"].sort_values(by="user_id"))
merged = df_filter.reset_index().merge(df_filter.reset_index(), on='user_id')
merged = merged[merged.index_x != merged.index_y]
gp2 = merged.groupby(["end_site_x", "end_site_y"]).agg(UniqueJourneys=("journey_id_y", "nunique"), UniqueUsers=("user_id", 'nunique'))
#gp2["AvgVisits"] = gp2["UniqueJourneys"] / gp2["UniqueUsers"]

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(gp2.sort_values(by=["end_site_x", "UniqueJourneys"], ascending=[True, False]))
#display(gp2[gp2.index.isin(["Arena Paris Nord"], level=0)].sort_values(by=["end_site_x", "UniqueJourneys"], ascending=[True, False]))
# display(df_filter[df_filter['user_id'] == 1117])
# display(merged[merged['user_id'] == 1117])

# display(df_filter[df_filter['user_id'] == 30369])
# display(merged[merged['user_id'] == 30369])

In [ ]:
# import branca.colormap as cm
# colormap = cm.LinearColormap(["green", 'yellow', "red"], vmin=1, vmax=gp["UniqueUsers"].max())
# unstack = gp["UniqueUsers"].unstack()
# def color(val):
#     if not np.isnan(val):
#         return 'background-color: %s' % colormap(val)
#     return "background-color: lightgray"
# unstack.style.map(color).format(precision=0)

In [ ]:
df_filter

In [ ]:
totals = df_filter.groupby("end_site").agg(TotalUniqueJourneys=("journey_id", "nunique"), TotalUniqueUsers=("user_id", 'nunique'), MeanTripDurationSeconds=("duration", "median"))
totals

In [ ]:
# Convert columns to datetime
df_src['start_time'] = pd.to_datetime(df_src['start_time'], format="mixed")
df_src['end_time'] = pd.to_datetime(df_src['end_time'], format="mixed")

# Calculate duration in seconds
df_src['duration'] = (df_src['end_time'] - df_src['start_time']).dt.total_seconds()

agg_duration = df_src.groupby(['journey_id', 'transportation_mode_tr']).agg(
    JourneyTransportDuration=('duration', 'sum')
).reset_index()
#agg_duration
# # Sort the duration aggregation and find the top two transportation modes for each journey
# agg_duration_sorted = agg_duration.sort_values(by=['journey_id', 'total_duration'], ascending=[True, False])

# # Get the top transportation modes for each journey
# agg_duration_top = agg_duration_sorted.groupby('journey_id').head(1).reset_index(drop=True)
# agg_duration_top

In [ ]:
merge_back = df_filter.merge(agg_duration, how='left', on="journey_id")
mb = merge_back.groupby(["end_site", "transportation_mode_tr"]).agg(MeanTransportDuration=("JourneyTransportDuration", 'median'), MeanJourneyDuration=("duration", 'median'), Count=("journey_id", "count"))
mb_sorted = mb.sort_values(by=["end_site","Count"], ascending=[True, False]).reset_index()
gp3 = mb_sorted[mb_sorted["transportation_mode_tr"] != "FOOT"].groupby('end_site').head(1)

In [ ]:
gp

In [ ]:
gp2

In [ ]:
gp3

In [ ]:
result = {}

for index, row in gp.reset_index().iterrows():
    end_site = row['end_site']
    previous_site = row['previous_site']
    total_occurrence = row['TotalOccurence']
    mean_duration = row['MeanDuration']
    
    if end_site not in result:
        result[end_site] = {}
    
    result[end_site][previous_site] = {
        'times_this_was_the_previous_visited_site': total_occurrence,
        'mean_duration_between_visits_in_hours': mean_duration.total_seconds() / 60 / 60
    }
    if np.isnan(result[end_site][previous_site]['mean_duration_between_visits_in_hours']):
        result[end_site][previous_site]['mean_duration_between_visits_in_hours'] = 0

for index, row in gp2.reset_index().iterrows():
    end_site = row['end_site_x']
    other_site = row['end_site_y']
    
    if end_site not in result:
        result[end_site] = {}
    
    if other_site not in result[end_site]:
        result[end_site][other_site] = {}
    
    result[end_site][other_site]["times_this_was_also_visited"] = row["UniqueJourneys"]
    result[end_site][other_site]["unique_users_who_also_visited"] = row["UniqueUsers"]

for index, row in totals.reset_index().iterrows():
    end_site = row['end_site']
    
    if end_site not in result:
        result[end_site] = {}
    
    result[end_site]["total_unique_journeys"] = row["TotalUniqueJourneys"]
    result[end_site]["total_unique_users"] = row["TotalUniqueUsers"]
    result[end_site]["average_visits_per_user"] = row["TotalUniqueJourneys"] / row["TotalUniqueUsers"]
    result[end_site]["mean_trip_duration_seconds"] = row["MeanTripDurationSeconds"]

for index, row in gp3.reset_index().iterrows():
    end_site = row['end_site']
    
    if end_site not in result:
        result[end_site] = {}
    
    result[end_site]["most_common_transport"] = row["transportation_mode_tr"]
    result[end_site]["most_common_transport_total_unique_journeys"] = row["Count"]
    result[end_site]["mean_trip_duration_seconds_including_most_common_transport"] = row["MeanJourneyDuration"]
    result[end_site]["mean_trip_duration_seconds_with_most_common_transport"] = row["MeanTransportDuration"]


In [ ]:
with open(OUTPUT_SITE_BEHAVIOR_FILE, 'w') as f:
    f.write(json.dumps(result, indent=4))